## Importing the modules.

In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Creating the get_url function to get the url.

In [22]:
def get_url(brand, location):

    brand=brand.lower()
    # Convert location to lowercase for case-insensitive comparison
    location_lower = location.lower()

    # Dictionary mapping location names to their IDs
    locations = {
        'delhi ncr': 1,
        'new delhi': 2,
        'mumbai': 2378,
        'bangalore': 5709,
        'hyderabad': 3686,
        'ahmedabad': 1692,
        'gurgaon': 5,
        'chennai': 5732,
        'pune': 2423,
        'noida': 134,
        'ghaziabad': 132,
        'lucknow': 290,
        'jaipur': 2130,
        'kolkata': 777,
        'kochi': 6356,
        'indore': 2920
    }

    # Check if the location exists in the dictionary
    if location_lower in locations:
        location_id = locations[location_lower]
    else:
        raise ValueError(f"Location '{location}' is not supported.")
    
    #defining the base url
    base_url = f'https://www.cars24.com/buy-used-car?f=make%3A%3D%3A{brand}&sort=bestmatch&serveWarrantyCount=true&listingSource=TabFilter&storeCityId={location_id}'
    
    return base_url


## Creating the data_extraction function to extract the details of the cars.

In [24]:
def data_extraction(brand, location):
    try:
        url = get_url(brand, location)
        
        # Make a request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract car names and years
            car_names = soup.find_all("h3", {"class": "_11dVb"})
            cars = []
            years = []
            for i in car_names:
                parts = i.text.split()
                years.append(parts[0])
                cars.append(" ".join(parts[1:]))
            
            # Extract distance traveled, fuel type, transmission type
            data2 = soup.find_all("div", {"class": "_2YB7p"})
            dist = []
            fuel = []
            trans = []
            for i in data2:
                data = i.find_all('li')
                dist.append(data[0].text)
                fuel.append(data[2].text)
                trans.append(data[4].text)
            
            # Extract prices
            prices = [i.text for i in soup.find_all('strong', {"class": "_3RL-I"})]
            
            # Create a dictionary to store the extracted data
            scraped_data = {
                'Car Model': cars,
                'Year of Manufacture': years,
                'Distance Traveled': dist,
                'Fuel Type': fuel,
                'Transmission Type': trans,
                'Prices': prices
            }
            
            return scraped_data
        
        else:
            print(f"Failed to retrieve data for {brand} in {location}. Status code: {response.status_code}")
            return None
    
    except Exception as e:
        print(f"An error occurred for {brand} in {location}: {e}")
        return None

In [25]:
output=data_extraction("Mahindra","mumbai")

In [27]:
result=pd.DataFrame(output)

In [28]:
result

,Car Model,Year of Manufacture,Distance Traveled,Fuel Type,Transmission Type,Prices
0,Mahindra XUV300 W6 1.5 DIESEL,2022,"16,454 km",Diesel,Manual,₹10.45 Lakh
1,Mahindra XUV300 W8 (O) 1.2 PETROL,2019,"69,558 km",Petrol,Manual,₹7.71 Lakh
2,Mahindra XUV300 W6 1.2 PETROL AMT,2022,"13,010 km",Petrol,Automatic,₹9.62 Lakh
3,Mahindra XUV300 W8 1.5 DIESEL,2019,"80,839 km",Diesel,Manual,₹8.28 Lakh
4,Mahindra XUV500 W10 AT,2017,"57,449 km",Diesel,Automatic,₹11.38 Lakh
5,Mahindra XUV300 W6 1.5 DIESEL AMT,2021,"38,524 km",Diesel,Automatic,₹9.28 Lakh
6,Mahindra XUV500 W10 AT,2017,"84,784 km",Diesel,Automatic,₹10.44 Lakh
7,Mahindra XUV300 W8 1.5 DIESEL,2019,"51,938 km",Diesel,Manual,₹8.84 Lakh
8,Mahindra XUV300 W6 1.2 PETROL,2019,"39,094 km",Petrol,Manual,₹7.89 Lakh
9,Mahindra XUV300 W8 (O) 1.2 PETROL AMT,2022,"20,405 km",Petrol,Automatic,₹11.27 Lakh
